<a href="https://colab.research.google.com/github/dimicorn/bubbles/blob/master/bubble_short.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from scipy import integrate
import numpy as np
import openpyxl
import matplotlib.pyplot as plt

In [8]:
# All values of gamma
G = [1.001, 1.01, 1.2, 1.3, 1.4, 1.5, 1.6, 5/3, 1.7, 1.9, 2, 4]

# Same for k_rho
K = [0, 0.5, 1, 1.5, 2, 2.5, 3]

# And n_int too
N_int = [0, 0.5, 1, 1.5, 2, 2.5, 3]

# Colors for plot option
Colors = ['purple', 'blue', 'red', 'green', 'cyan', 'orange', 'black', 'gray']

# Starting and ending values of lambda
x0 = 1
x_k = 0.7

# Initial conditions
v0 = 1
rho0 = 1
p0 = 1

# Plot labels
Gamma = 'gamma = '
K_rho = 'k_rho = '
N = 'n_int = '

In [9]:
class Bubble(object):
    def __init__(self, a, b, c):
        self.gamma = a
        self.k_rho = b
        self.n_int = c
        g = self.gamma
        k = self.k_rho

        # Solving system of ODE's
        def solver(x, r):
            v, rho, p = r
            n1 = self.n_int
            n = (2 + n1) / (5 - k)
            fv = (4 * g * v / ((g + 1) * x) - k - (1 - 1 / n) * (
                    ((g + 1) / (g - 1)) * (2 * v / (g + 1) - x) * rho * v / p - 2)) * (
                         ((g + 1) / (g - 1)) * (2 * v / (g + 1) - x) ** 2 * rho / p - 2 * g / (g + 1)) ** (-1)
            fp = ((1 / n - 1) * v - (2 * v / (g + 1) - x) * fv) * ((g + 1) / ((g - 1) * rho))
            f_rho = ((-k * (g - 1) - 2 * (1 - 1 / n)) * (2 * v / (g + 1) - x) ** (-1) - 1 / p * fp) * (-rho / g)
            return fv, f_rho, fp

        # Solutions for the system
        sol = integrate.solve_ivp(solver, (x0, x_k), (v0, rho0, p0),
                                  t_eval=np.linspace(x0, x_k, 1000), method='Radau')
        self.velocity, self.density, self.pressure = sol.y
        self.lambda_c = sol.t
        self.slope_x = np.linspace(x0, x_k, 1000)
        self.slope_y = np.array([(self.gamma + 1) / 2 * self.slope_x[e] for e in range(1000)])
        self.dist = -1
        self.intersect = False
        x1, y1, x2, y2 = self.lambda_c, self.velocity, self.slope_x, self.slope_y
        count = 0
        for i in range(len(x1)):
            for j in range(len(x2)):
                if x1[i] == x2[j] and y1[i] == y2[j]:
                    count += 1
                    self.dist = 0
        if count == 0:
            lamb, vel = self.lambda_c[-1], self.velocity[-1]
            s = (lamb - 2 / (g + 1) * vel) * (2 * (g + 1)) / (4 + (g + 1) ** 2)
            ox = lamb - s * (g + 1) / 2
            oy = ox * (g + 1) / 2
            d = np.sqrt((ox - lamb) ** 2 + (oy - vel) ** 2)
            self.dist = d
        if self.dist < 0.001:
            self.intersect = True

    # Function for eta
    def eta(self):
        k_rho = self.k_rho
        n_int = self.n_int
        x = (2 + n_int) / (5 - k_rho)
        return x

    # Value of the curve at lambda_c
    def v2(self, x, y):
        z = (x + 1) / 2 * y
        return z

    # Approximation using eqn B8a
    def lambda2(self, x, k_rho):
        w = self.eta()
        t = x ** 3 + 12 * x ** 2 + 8 * x + 1 - 0.5 * (x + 1) * (3 * x + 1) * k_rho - (x + 1) * (4 * x + 1) / w
        u = 2 * x ** 3 + 12 * x ** 2 + 7 * x + 1 - 0.5 * (x + 1) * (3 * x + 1) * k_rho - (x + 1) * (4 * x + 1) / w
        return t / u

    # Gradient of velocity, r = R_s
    def dv1(self, x, k_rho):
        w = self.eta()
        y = (-(7 * x + 3) + (x + 1) * k_rho + 3 * (x + 1) / w) / (x + 1)
        return y

    # Gradient of velocity, r = R_c
    def dv2(self, x, k_rho):
        w = self.eta()
        y = (-2 * (x + 1) + k_rho + 2 / w) / (2 * x / (x + 1))
        return y

    # Difference between lambda from the ODE and the approximation using eqn B8a
    def delta(self, x, y):
        z = abs(x - y) / x * 100
        return z

    # Calculating R_sw
    def r_sw(self, v, numb, k):
        g = self.gamma
        f_rho = (4 * g / (g + 1) ** 2) ** (1 / (g - 1))
        t_y = 3600 * 24 * 365
        t = numb * t_y
        n = (2 + self.n_int) / (5 - self.k_rho)
        r = (g - 1) / (g + 1) * v * f_rho * t * (3 * g / (3 * (g - 1) * n + self.n_int)) / k ** 3
        return r / au

    # Adding all the necessary data to the excel table
    def values(self, sh1, count):
        d = self.dist
        if self.intersect:
            sh1['A' + str(count)].value = self.gamma
            sh1['B' + str(count)].value = self.k_rho
            sh1['C' + str(count)].value = self.n_int
            l2 = self.lambda2(self.gamma, self.k_rho)
            sh1['D' + str(count)].value = self.delta(self.lambda_c[-1], l2)
            sh1['E' + str(count)].value = d
            sh1['F' + str(count)].value = self.q_p()

    # Scaling for the velocity curves
    def norm1(self):
        lamb, vel, pres = self.lambda_c, self.velocity, self.pressure
        for t in range(len(lamb)):
            lamb[t] = 1 - (1 - lamb[t]) / (self.gamma - 1)
        for t in range(len(vel)):
            vel[t] = 1 + (vel[t] - 1) / (self.gamma - 1)
        return lamb, vel

    # Scaling for the linear slope
    def norm2(self):
        g = self.gamma
        a = self.slope_x
        b = self.slope_y
        for t in range(len(a)):
            a[t] = 1 - (1 - a[t]) / (g - 1)
        for t in range(len(b)):
            b[t] = 1 + (b[t] - 1) / (g - 1)
        return a, b

    # Values of Q_p
    def q_p(self):
        g = self.gamma
        p_r = 2 / (g + 1) * self.pressure[-1]
        p_sw = (2/(g + 1)) * ((g + 1)**2/(4*g))**(g/(g-1))
        return p_sw/p_r



# Repeating plot legend for pressure
def leg():
  plt.plot([], [], 'k-', label=N + str(N_int[0]))
  plt.plot([], [], 'k--', label=N + str(N_int[1]))
  plt.plot([], [], 'k-.', label=N + str(N_int[2]))
  plt.plot([], [], 'k.', label=N + str(N_int[3]))


# Repeating plot label
def lab(y):
  plt.grid()
  plt.xlabel('Lambda')
  plt.ylabel('P')
  plt.title('P(Lambda), ' + Gamma + str(G[y]))
  plt.legend()
  plt.savefig('P(Lambda), ' + Gamma + str(G[y]) + '.png', dpi=300)
  plt.show()

In [25]:
K_ext = np.linspace(0, 3, num=20)
N_ext = np.linspace(0, 3, num=20)

def main():
  intersects = np.full((len(G), len(K_ext), len(N_ext)), -1)
  Qp_value = np.full((len(G), len(K_ext), len(N_ext)), -1.0)

  for i in range(len(G)):
    for j in range(len(K_ext)):
      for q in range(len(N_ext)):
        bubble = Bubble(G[i], K_ext[j], N_ext[q])
        intersects[i][j][q] = bubble.intersect
        Qp_value[i][j][q] = bubble.q_p()
        print(G[i], K_ext[j], N_ext[q], intersects[i][j][q], Qp_value[i][j][q])
        '''
        if bubble.intersect:
          q_p = bubble.q_p()
          if 0.9 < q_p < 0.95:
            plt.scatter(K_ext[j], N_ext[q], c='cyan')
          elif 0.95 < q_p < 1.05:
            plt.scatter(K_ext[j], N_ext[q], c='blue')
          elif 1.05 < q_p < 1.15:
            plt.scatter(K_ext[j], N_ext[q], c='darkcyan')
          else:
            plt.scatter(K_ext[j], N_ext[q], c='red')
        else:
          plt.scatter(K_ext[j], N_ext[q], c='gray')
    plt.plot([], [], c='cyan', marker='o', label='0.9 < Q_p < 0.95')
    plt.plot([], [], c='blue', marker='o', label='0.95 < Q_p < 1.05')
    plt.plot([], [], c='darkcyan', marker='o', label='1.05 < Q_p < 1.15')
    plt.title('Colormap, ' + Gamma + str(G[i]))
    plt.xlabel('K_rho')
    plt.ylabel('N_int')
    # plt.legend()
    plt.savefig('Colormap, ' + Gamma + str(G[i]) + '.png', dpi=300)
    '''
  np.save('intersects.npy', intersects)
  np.save('Qp_value.npy', Qp_value)

In [24]:
main()


1.001 0.0 0.0 1 1.000250031221442
1.001 0.0 0.15789473684210525 1 1.000250031221442
1.001 0.0 0.3157894736842105 1 1.000250031221442
1.001 0.0 0.47368421052631576 1 1.000250031221442
1.001 0.0 0.631578947368421 1 1.000250031221442
1.001 0.0 0.7894736842105263 1 1.000250031221442
1.001 0.0 0.9473684210526315 1 1.000250031221442
1.001 0.0 1.1052631578947367 1 1.000250031221442
1.001 0.0 1.263157894736842 1 1.000250031221442
1.001 0.0 1.4210526315789473 1 1.000250031221442
1.001 0.0 1.5789473684210527 1 1.000250031221442
1.001 0.0 1.7368421052631577 1 1.000250031221442
1.001 0.0 1.894736842105263 1 1.000250031221442
1.001 0.0 2.052631578947368 1 1.000250031221442
1.001 0.0 2.2105263157894735 1 1.000250031221442
1.001 0.0 2.3684210526315788 1 1.000250031221442
1.001 0.0 2.526315789473684 1 1.000250031221442
1.001 0.0 2.6842105263157894 1 1.000250031221442
1.001 0.0 2.8421052631578947 1 1.000250031221442
1.001 0.0 3.0 0 1.6805506184942112
1.001 0.15789473684210525 0.0 1 1.000250031221442
1.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in double_scalars


1.001 0.47368421052631576 2.526315789473684 0 1.9008275510300046
1.001 0.47368421052631576 2.6842105263157894 1 1.000250031221442
1.001 0.47368421052631576 2.8421052631578947 1 1.000250031221442
1.001 0.47368421052631576 3.0 1 1.000250031221442
1.001 0.631578947368421 0.0 1 1.000250031221442
1.001 0.631578947368421 0.15789473684210525 1 1.000250031221442
1.001 0.631578947368421 0.3157894736842105 1 1.000250031221442
1.001 0.631578947368421 0.47368421052631576 1 1.000250031221442
1.001 0.631578947368421 0.631578947368421 1 1.000250031221442
1.001 0.631578947368421 0.7894736842105263 1 1.000250031221442
1.001 0.631578947368421 0.9473684210526315 1 2.928381761774601
1.001 0.631578947368421 1.1052631578947367 1 1.9045920310873108
1.001 0.631578947368421 1.263157894736842 1 1.5631847995571584
1.001 0.631578947368421 1.4210526315789473 1 1.000250031221442
1.001 0.631578947368421 1.5789473684210527 1 1.000250031221442
1.001 0.631578947368421 1.7368421052631577 1 1.000250031221442
1.001 0.6315